# Assignment 3

## Question 1

```
INPUT SENTENCE 1 I love apples
VADER OUTPUT {'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369}

```
#### There is a correct interpretation that the sentence is positive, given by the fact that it contains the word 'love', which in the lexicon has a sentiment rating of 3.2 (close to 4), which indicates a positive sentiment.

```
INPUT SENTENCE 2 I don't love apples
VADER OUTPUT {'neg': 0.627, 'neu': 0.373, 'pos': 0.0, 'compound': -0.5216}
```
#### There is a correct interpretation that the sentence is a little negative and neutral, given by the fact that it contains the word 'love', which in the lexicon has a sentiment rating of 3.2 (close to 4), but also has the negation "don't". This negation inverts the polarity of "love", that is, although "love" is a positive word, it is interpreted as negative one. 

```
INPUT SENTENCE 3 I love apples :-)
VADER OUTPUT {'neg': 0.0, 'neu': 0.133, 'pos': 0.867, 'compound': 0.7579}
```
#### There is a correct interpretation that it is highly positive. It recognizes the word "love" as highly positive in the lexicon and also considers the presence of the emoticon ":-)" as an intensifier, contributing to the overall positive sentiment.
```

INPUT SENTENCE 4 These houses are ruins
VADER OUTPUT {'neg': 0.492, 'neu': 0.508, 'pos': 0.0, 'compound': -0.4404}

```
#### Here, VADER is a little unsure wether the sentence is negative or neutral, although not positive, which is good. That is because "ruins" is considered in the lexicon as negative (-1.9), but as there are no other negative intensifiers, it is more neutral.

INPUT SENTENCE 5 These houses are certainly not considered ruins
VADER OUTPUT {'neg': 0.0, 'neu': 0.51, 'pos': 0.49, 'compound': 0.5867}

#### Here, VADER gives a correct interpretation of the sentence as being positive and neutral. That is because, although it recognizes "ruins" as being negative, it turns it to positive because of the intensified negator "certainly not". As there are no more intensifiers of sentiment, the sentence also oscilates torwards a neutral sentiment.

```
INPUT SENTENCE 6 He lies in the chair in the garden
VADER OUTPUT {'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.4215}

```
#### Here, there is a bit of misinterpretation. Although the sentence is completely neutral, given that is is just an information, VADER interprets the word "lies" as negative in its lexicon. That is because this word is a homonym to the infinitive verb "lies", which means "to not tell the truth", which induces a negative sentiment. This makes it interpret the sentence as having 28.6% of negative content, although it should be 100% neutral.

```
INPUT SENTENCE 7 This house is like any house
VADER OUTPUT {'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'compound': 0.3612
```

#### Here, VADER made the same mistake as INPUT SENTENCE 6, but instead of negative, it interpreted the sentence as being one third positive, although it should also be completely neutral. That is because it recognizes the word "like" as having a positive valence score in its lexicon, because it is a homonym to the verb "like" as in "to like something".

## Question 2


In [1]:
import json
my_tweets = json.load(open('my_tweets.json'))

In [2]:
for id_, tweet_info in my_tweets.items():
    print(id_,tweet_info)
    break

1 {'sentiment_label': 'negative', 'text_of_tweet': 'I hate when pretty girls act mean cause bitch now you’re ugly.', 'tweet_url': 'https://twitter.com/raveneliseee/status/1763188901617819703'}


## Question 3

In [3]:
def vader_output_to_label(vader_output):
    
    
    """
    map vader output e.g.,
    {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4215}
    to one of the following values:
    a) positive float -> 'positive'
    b) 0.0 -> 'neutral'
    c) negative float -> 'negative'
    
    :param dict vader_output: output dict from vader
    
    :rtype: str
    :return: 'negative' | 'neutral' | 'positive'
    """
    compound = vader_output['compound']
    
    if compound < 0:
        return 'negative'
    elif compound == 0.0:
        return 'neutral'
    elif compound > 0.0:
        return 'positive'
    
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.0}) == 'neutral'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.01}) == 'positive'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': -0.01}) == 'negative'

In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy
nlp = spacy.load('en_core_web_sm')

vader_model = SentimentIntensityAnalyzer()

tweets = []
all_vader_output = []
gold = []

# settings (to change for different experiments)
to_lemmatize = True 
pos = set()

def run_vader(textual_unit, 
              parts_of_speech_to_consider=None, lemmatize=False,
              verbose=0):
    """
    Run VADER on a sentence from spacy
    
    :param str textual unit: a textual unit, e.g., sentence, sentences (one string)
    (by looping over doc.sents)
    :param bool lemmatize: If True, provide lemmas to VADER instead of words
    :param set parts_of_speech_to_consider:
    -None or empty set: all parts of speech are provided
    -non-empty set: only these parts of speech are considered.
    :param int verbose: if set to 1, information is printed
    about input and output
    
    :rtype: dict
    :return: vader output dict
    """
    doc = nlp(textual_unit)
        
    input_to_vader = []

    for sent in doc.sents:
        for token in sent:

            to_add = token.text

            if lemmatize:
                to_add = token.lemma_

                if to_add == '-PRON-': 
                    to_add = token.text

            if parts_of_speech_to_consider:
                if token.pos_ in parts_of_speech_to_consider:
                    input_to_vader.append(to_add) 
            else:
                input_to_vader.append(to_add)

    scores = vader_model.polarity_scores(' '.join(input_to_vader))
    
    if verbose >= 1:
        print()
        print('INPUT SENTENCE', sent)
        print('INPUT TO VADER', input_to_vader)
        print('VADER OUTPUT', scores)

    return scores

for id_, tweet_info in my_tweets.items():
    the_tweet = tweet_info['text_of_tweet']
    vader_output = run_vader(the_tweet,lemmatize=to_lemmatize)# run vader
    vader_label = vader_output_to_label(vader_output)# convert vader output to category
    
    tweets.append(the_tweet)
    all_vader_output.append(vader_label)
    gold.append(tweet_info['sentiment_label'])
    
# use scikit-learn's classification report

### 3.a

In [5]:
total_tweets = len(tweets)
wrong_guesses = {}

for i in range(total_tweets):
    print("tweet:",tweets[i])
    print("vader label:", all_vader_output[i])
    print("gold label:", gold[i])
    if all_vader_output[i] != gold[i]:
        category_tweets_dict = wrong_guesses.get(gold[i],{})
        wrong_tweets_by_category = category_tweets_dict.get(all_vader_output[i],[])
        wrong_tweets_by_category.append(tweets[i])
        category_tweets_dict[all_vader_output[i]] = wrong_tweets_by_category
        wrong_guesses[gold[i]] = category_tweets_dict
        
    print()

tweet: I hate when pretty girls act mean cause bitch now you’re ugly.
vader label: negative
gold label: negative

tweet: I hate when people take kindness for granted
vader label: positive
gold label: negative

tweet: I really hate the fact that some people don’t have the patience to read and understand basic communication & instructions……  it’s unfair to humanity.
vader label: negative
gold label: negative

tweet: i hate feeling like i’m begging someone to do something they said they would. just keep your word please.
vader label: positive
gold label: negative

tweet: People who do things and blame it on alcohol disgust me
vader label: negative
gold label: negative

tweet: People that are love obsessed disgust tf out of me, cause why do you always want to be in love(a relationship) are you possessed?
vader label: negative
gold label: negative

tweet: Your friend is literally shtting on your interest to your face after you expressed discomfort… I doubt theyd care this much about you
vad

In [6]:
from sklearn.metrics import classification_report

report = classification_report(gold,all_vader_output)
print(report)

              precision    recall  f1-score   support

    negative       0.65      0.73      0.69        15
     neutral       0.80      0.25      0.38        16
    positive       0.61      0.89      0.72        19

    accuracy                           0.64        50
   macro avg       0.68      0.63      0.60        50
weighted avg       0.68      0.64      0.60        50



#### Having a look at the metrics, this is what we can extract:
* VADER has a good precision (80%) on neutral sentiment data, which means that most of the time it guesses the sentiment as being neutral, it is correct. On the other hand, it has a poor performance when we consider the recall, that is, a large number of sentences which were neutral were either recognized as being positive or negative. This indicates that it has to be trained more on neutral data. This might have happened because the neutral training data is largely composed of scientific news headlines, and sometimes the subject of the article can be either interpreted as negative (such as diseases) or positive (such as health treatment improvements).
* VADER also shows a good recall on positive data. However, the precision is a bit low. This indicates that although it almost does not miss a positive labeled tweet, it assumes a good part of the tweets were positive when in fact they were neutral or negative. This indicates the need to improve training on positive sentiment data.
* Lastly, VADER shows a similar performance in negative data as it did on positive, when it comes to precision. However, the recall is significantly worse, making it clear that there was a good part of tweets which were negative but were not recognized.

#### Although the metrics observed gave hints about what needs improvement, in this classification problem it is unclear wether we need to improve recall or precision. That is because there is no consequence whatsoever of incorectly guessing a category.

### 3.b

### Negative

In [7]:
for category,tweets in wrong_guesses["negative"].items():
    print(f"Tweets guessed as {category}:\n")
    for tweet in tweets:
        print(tweet)
print("---")

Tweets guessed as positive:

I hate when people take kindness for granted
i hate feeling like i’m begging someone to do something they said they would. just keep your word please.
Your friend is literally shtting on your interest to your face after you expressed discomfort… I doubt theyd care this much about you
Nothing worse than a woman who blames her bad behavior on her zodiac sign
---


#### In the sentences that VADER guessed as being negative, they were all incorrectly guessed as positive. In the first one, this could be attributed to the fact that it also contains the word "kindness", which has a really positive valence on the lexicon (2.3), and maybe that could be bringing the score up. Apart from the third sentence, which has the word "friend", all other tweets are guessed really innaccurately. That maybe can be caused by grammatical erros and poor harmony with the lexicon, so that VADER has not much data out of a sentence to correctly guess what sentiment it is.

### Positive

In [8]:
for category,tweets in wrong_guesses["positive"].items():
    print(f"Tweets guessed as {category}:\n")
    for tweet in tweets:
        print(tweet)
    print()
print("---")

Tweets guessed as negative:

if i ever get to hear him sing everythingoes in person, all my discomfort would wash away

Tweets guessed as neutral:

Gm legends 💜
Now, this guy is the definition of humble. What a legend 👊

---


#### The first sentence is guessed as negative. That can be attributed to the fact that it contains the work "discomfort", which is a negative word according to the lexicon. The other sentence is guessed as neutral, and this can maybe due to the fact that VADER does not recognize "the definition of humble" as a compliment, maybe because "humble" is not in the lexicon, although this sentence contains emojis which could give a hint to the sentence being positive.

### Neutral

In [9]:
for category,tweets in wrong_guesses["neutral"].items():
    print(f"Tweets guessed as {category}:\n")
    for tweet in tweets:
        print(tweet)
    print()
print("---")

Tweets guessed as positive:

Here’s a look at some of the highlights worth watching from late-night TV.
Be honest, does banana bread taste nice to you?
Deep brain stimulation (DBS) maps dysfunctional frontal cortical circuits in dystonia, Parkinson's disease, Tourette's syndrome, and OCD, which helped to guide stimulation sites in other patients
@b_hollunder NingfeiL @andreashorn_ @netstim_org
As impressive as they seem, the latest computer brains usually fail at tasks that require generalizing concepts — tasks that come easily to humans.
A retrovirus — embedded in the DNA of jawed vertebrates — helps turn on production of a protein needed to insulate nerve fibers. Such insulation, called myelin, may have helped make speedy thoughts and complex brains possible.
Today, artificial intelligence can identify potential drug targets and develop medicines from scratch. But whether the approach can craft drugs that help patients remains to be seen.
OpenAI wants to make a walking, talking human

#### We can see that the neutral sentiment is the hardest one for VADER to guess precisely. That can be attributed to the fact that it takes the words inside the headlines (such as "isn't good", "smarter", "impressive") and retrieve their sentiment from the lexicon. This affects the sentence as a whole, as there are not many intensifiers or other attributes which make the sentiment of the sentence clear.

## Question 4

In [10]:
from sklearn.datasets import load_files
import pathlib

In [11]:
cwd = pathlib.Path.cwd()

In [12]:
airline_tweets_folder = cwd.joinpath('airlinetweets')

In [13]:
print('this will print True if the folder exists:', 
      airline_tweets_folder.exists())

this will print True if the folder exists: True


In [16]:
dataset = load_files(airline_tweets_folder)

In [25]:
print(dataset.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


* Run VADER (as it is) on the set of airline tweets 

In [37]:
gold = []
for target in dataset.target:
    gold.append(dataset.target_names[target])

vader_output = []

for tweet in dataset.data:
    scores = run_vader(str(tweet),lemmatize=to_lemmatize)
    vader_output.append(vader_output_to_label(scores))

report = classification_report(gold,vader_output)
print(report)

ok
              precision    recall  f1-score   support

    negative       0.79      0.52      0.63      1750
     neutral       0.60      0.49      0.54      1515
    positive       0.56      0.88      0.68      1490

    accuracy                           0.62      4755
   macro avg       0.65      0.63      0.62      4755
weighted avg       0.65      0.62      0.62      4755

